## 1. 데이터 로딩

샘플로 1000 개의 뉴스 기사만 이용합니다. 

In [1]:
import config

from navernews_10days import get_news_paths
from soynlp.utils import DoublespaceLineCorpus

path = get_news_paths(tokenize='komoran', date='2016-10-20')
corpus = DoublespaceLineCorpus(path, iter_sent=True, num_sent=1000)

soynlp=0.0.49
added lovit_textmining_dataset


## 2. Build bigram dictionary

bigram_extractor 는 min_count 이상인 bigram 만을 선택하는 extractor 입니다. 

to_bigram 은 두 개의 list 를 zip 으로 묶어서 bigram list 을 만드는 함수입니다. 

In [2]:
from collections import Counter

def bigram_extractor(sents, min_count=10):

    def to_bigram(tokens):
        bigrams = [(t0, t1) for t0, t1 in zip(tokens, tokens[1:])]
        return bigrams

    unigram_counter = Counter(word for sent in sents for word in sent.split())
    bigram_counter = Counter(
        (bigram for sent in sents for bigram in to_bigram(sent.split()))
    )

    bigram_dictionary = {
        bigram:count for bigram, count in bigram_counter.items()
        if count >= min_count and '/NN' in bigram[0]
    }

    def score(bigram, count):
        a = unigram_counter[bigram[0]]
        b = unigram_counter[bigram[1]]
        s = (count - min_count) / (a * b)
        return s

    bigram_score = {
        bigram:score(bigram, count) for bigram, count in bigram_dictionary.items()
    }

    return bigram_score

bigrams = bigram_extractor(corpus)
len(bigrams)

113

bigram 을 살펴봅니다. 

In [3]:
sorted(bigrams.items(), key=lambda x:-x[1])[5:15]

[(('자료/NNG', '사진/NNP'), 0.010714285714285714),
 (('국제/NNG', '사회/NNG'), 0.008415147265077139),
 (('확장/NNG', '억제/NNG'), 0.0084090600840906),
 (('공동/NNG', '성명/NNG'), 0.007918552036199095),
 (('핵/NNG', '미사일/NNG'), 0.006572295247724975),
 (('미사일/NNG', '위협/NNG'), 0.006521739130434782),
 (('외교/NNG', '국방장관/NNG'), 0.005853994490358127),
 (('외교/NNG', '국방/NNP'), 0.004662004662004662),
 (('억제/NNG', '전략/NNP'), 0.004641089108910891),
 (('수상전/NNP', '센터/NNP'), 0.004201680672268907)]

## 3. Bigram tokenizer

BigramTokenizer 는 base tokenizer 인 tagger 와 bigrams 를 입력받는 class 입니다. 

\_\_call\_\_() 함수를 구현하면 클래스도 함수처럼 호출할 수 있습니다. 

In [4]:
class BigramTokenizer:

    def __init__(self, bigrams, tokenize=lambda x:x.split()):
        self.bigrams = bigrams
        self.tokenize = tokenize

    def __call__(self, sent):
        if not sent:
            return []

        unigrams = self.tokenize(sent)

        bigrams = [(t0, t1) for t0, t1 in zip(unigrams, unigrams[1:])]
        bigrams = [bigram for bigram in bigrams if bigram in self.bigrams]
        bigrams = ['%s-%s' % (t0, t1) for t0, t1 in bigrams]

        return unigrams + bigrams

bigram_tokenizer = BigramTokenizer(bigrams)

샘플로 하나의 문장을 선택하여 토크나이징을 합니다. 

In [5]:
sent = "오패산터널/NNP 총격전/NNG 용의자/NNP 검거/NNG 서울/NNP 연합뉴스/NNP 경찰/NNG 관계자/NNG 들/XSN 이/JKS 19/SN 일/NNB 오후/NNG 서울/NNP 강북구/NNP 오/NNP 패사/NNG ㄴ/JX 터널/NNP 인근/NNG 에서/JKB 사제/NNP 총기/NNG 를/JKO 발사/NNG 하/XSV 아/EC 경찰/NNG 을/JKO 살해/NNG 하/XSV ㄴ/ETM 용의자/NNP 성모/NNP 씨/NNB 를/JKO 검거/NNG 하/XSV 고/EC 있/VV 다/EC 성씨/NNP 는/JX 검거/NNG 당시/NNG 서바이벌/NNP 게임/NNG 에서/JKB 쓰/VV 는/ETM 방탄/NNP 조끼/NNP 에/JKB 헬멧/NNP 까지/JX 착용/NNG 하/XSV ㄴ/ETM 상태/NNG 이/VCP 었/EP 다/EC 독자/NNG 제공/NNG 영상/NNP 캡처/NNP 연합뉴스/NNP"

bigram_tokenizer(sent) 로 클래스를 호출합니다. 

In [6]:
bigram_tokenizer(sent)

['오패산터널/NNP',
 '총격전/NNG',
 '용의자/NNP',
 '검거/NNG',
 '서울/NNP',
 '연합뉴스/NNP',
 '경찰/NNG',
 '관계자/NNG',
 '들/XSN',
 '이/JKS',
 '19/SN',
 '일/NNB',
 '오후/NNG',
 '서울/NNP',
 '강북구/NNP',
 '오/NNP',
 '패사/NNG',
 'ㄴ/JX',
 '터널/NNP',
 '인근/NNG',
 '에서/JKB',
 '사제/NNP',
 '총기/NNG',
 '를/JKO',
 '발사/NNG',
 '하/XSV',
 '아/EC',
 '경찰/NNG',
 '을/JKO',
 '살해/NNG',
 '하/XSV',
 'ㄴ/ETM',
 '용의자/NNP',
 '성모/NNP',
 '씨/NNB',
 '를/JKO',
 '검거/NNG',
 '하/XSV',
 '고/EC',
 '있/VV',
 '다/EC',
 '성씨/NNP',
 '는/JX',
 '검거/NNG',
 '당시/NNG',
 '서바이벌/NNP',
 '게임/NNG',
 '에서/JKB',
 '쓰/VV',
 '는/ETM',
 '방탄/NNP',
 '조끼/NNP',
 '에/JKB',
 '헬멧/NNP',
 '까지/JX',
 '착용/NNG',
 '하/XSV',
 'ㄴ/ETM',
 '상태/NNG',
 '이/VCP',
 '었/EP',
 '다/EC',
 '독자/NNG',
 '제공/NNG',
 '영상/NNP',
 '캡처/NNP',
 '연합뉴스/NNP',
 '서울/NNP-연합뉴스/NNP',
 '일/NNB-오후/NNG',
 '오/NNP-패사/NNG',
 '패사/NNG-ㄴ/JX',
 '인근/NNG-에서/JKB',
 '발사/NNG-하/XSV',
 '성씨/NNP-는/JX',
 '상태/NNG-이/VCP']

이를 CountVectorizer 의 tokenizer 로 입력합니다. 

In [7]:
from sklearn.feature_extraction.text import CountVectorizer

corpus.iter_sent=False
vectorizer = CountVectorizer(tokenizer=bigram_tokenizer)
x = vectorizer.fit_transform(corpus)

총 5,317 개의 uni + bigram 으로 term frequency vector 가 만들어집니다.

In [8]:
x.shape

(96, 5317)

## 4. All bigrams

sklearn.feature_extraction.text.CountVectorizer 의 ngram_range=(1, 2) 로 설정한다면, 가능한 모든 종류의 bigram 을 term 으로 이용하게 됩니다.

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

def all_bigram_tokenizer(sent):
    if not sent:
        return []
    return sent.split()

vectorizer_ = CountVectorizer(
    ngram_range=(1,2),
    tokenizer=all_bigram_tokenizer
)

x_ = vectorizer_.fit_transform(corpus)

25,748 개의 uni + bigram 을 이용하여 term frequency vector 를 만듭니다. ngram_range 를 이용할 때에는 min_df, max_df 를 이용하여 infrequent bigram 을 제거해야 적절한 크기의 차원을 유지할 수 있습니다.

In [10]:
x_.shape

(96, 25748)